In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

from datetime import datetime

from dask_utils import run_with_client

from common import *
from mcs_shared import SNOBAL_DIR, DATA_DIR, mcs_snotel_depth, x_y_snotel, SnotelPointData, mcs_snotel_csv, MC_ALS

## SNOTEL 

mcs_snotel_mt, mcs_snotel_point = mcs_snotel_depth(
    datetime(2020, 10, 1), datetime(2025, 7, 31)
)

In [ ]:
depth_2021 = pd.read_csv(
    MC_ALS + '/MCS-SNOTEL_2021.csv',
    parse_dates=['Date'], index_col=['Date'],
    comment='#',
)
depth_2021['MCS depth'] *= 0.0254

In [ ]:
mcs_snotel_point = SnotelPointData("637:ID:SNTL", "MCS")
utm_x, utm_y = x_y_snotel(mcs_snotel_point)

In [ ]:
mcs_snotel_mt = pd.concat([depth_2021, mcs_snotel_csv()])
mcs_snotel_mt.head()

## Model

In [ ]:
with run_with_client(24, 48):
    mcs_snobal_factor = xr.open_mfdataset(
        (SNOBAL_DIR / 'wy202[1,2,3,4,5]' / 'mcs/run202*/snow.nc').as_posix(),
        preprocess=lambda ds: ds['thickness'],
        parallel=True,
    ).sel(
        x=utm_x, 
        y=utm_y,
        method='nearest',
    ).compute()
   

In [ ]:
with run_with_client(24, 48):
    mcs_snobal_base = xr.open_mfdataset(
        (SNOBAL_DIR / 'wy202[1,2,3,4,5]' / 'mcs_base/run202*/snow.nc').as_posix(),
        preprocess=lambda ds: ds['thickness'],
        parallel=True,
    ).sel(
        x=utm_x, 
        y=utm_y,
        method='nearest',
    ).compute()

Filter to only midnight values

In [ ]:
mcs_snobal_factor = mcs_snobal_factor.sel(time=mcs_snobal_factor.time.dt.hour == 0)
# mcs_snobal_base = mcs_snobal_base.sel(time=mcs_snobal_base.time.dt.hour == 0)

# Matplotlib 

In [ ]:
import matplotlib.ticker as mticker
from matplotlib.offsetbox import AnchoredText

In [ ]:
STATION_LABEL = 'Station'
COLORS = {
        'iSnobal': 'cornflowerblue',
        'iSnobal (base)': 'goldenrod',
        STATION_LABEL: 'seagreen',    
}

def plot_site(data, ax, xlim):
    for key in data:
        if key == STATION_LABEL:
            continue

        ax.plot(
            data[key].time,
            data[key].data.flatten(),
            label=key, 
            color=COLORS[key], 
            alpha=0.9, lw=0.8
        )
    ax.plot(
        data[STATION_LABEL].index,
        data[STATION_LABEL].values,
        label=key, 
        color=COLORS[STATION_LABEL], 
        alpha=0.9, lw=1
    ) 

    ax.tick_params(axis='x', which='minor', tick1On=False, tick2On=False)
    ax.set_xlim([xlim[0], xlim[-1]])
    ax.tick_params(axis='x', which='major', pad=0, labelrotation=30)
    ax.xaxis.set_tick_params(labelsize=8)
    for tick in ax.xaxis.get_majorticklabels():
        tick.set_horizontalalignment("center")

    ax.set_ylabel(r'Snow Depth (m)', fontsize=8)
    ax.set_yticks(np.arange(0, 4))
    ax.set_ylim(top=3, bottom=-0.05)  
    ax.yaxis.set_minor_locator(mticker.MultipleLocator(0.2))

In [ ]:
periods = 58
plot_range = pd.date_range(start=f'{2021 - 1}-11-01', periods=periods, freq='MS')
xTicks = mdates.DateFormatter('%b-%Y')

figure_opts = dict(figsize=(8,3), dpi=300,)
fig, axes = plt.subplots(1, 1, sharex=True, **figure_opts)
plt.subplots_adjust(hspace=0.05)

plot_site(
    {
        'iSnobal': mcs_snobal_factor['thickness'],
        'iSnobal (base)': mcs_snobal_base['thickness'],
        STATION_LABEL: mcs_snotel_mt['MCS depth'],
    },
    axes,
    plot_range
)

plt.setp(axes.get_xticklabels()[-1], visible=False)

# at = AnchoredText(
#     f'Water Year {year} - 2022', 
#     prop=dict(size=10), 
#     frameon=False, 
#     loc='upper left', 
#     pad=0.3, 
#     borderpad=0.25,
# )
# axes.add_artist(at)
axes.legend(
    frameon=False,
    # bbox_to_anchor=(0.5, 1.09),
    loc='upper left',
    ncol=3,
    borderaxespad=0.15, 
    fontsize=8
);

# HV plot

## SNOTEL 

In [ ]:
use_hvplot()

In [ ]:
HV_PLOT_OPTS['ylabel'] = 'Snow Depth (m)'
y_lim = (-0.05, 3.)
SNOTEL_LINE=dict(line_width=3)

In [ ]:
mcs_snobal_factor.thickness.hvplot(label='iSnobal').opts(
    title='Mores Creek Summit', ylim=y_lim, **LINE_STYLE, **HV_PLOT_OPTS
) * \
mcs_snobal_base.thickness.hvplot(label='iSnobal (base)').opts(
    **LINE_STYLE
) * \
mcs_snotel_mt['MCS depth'].hvplot(label='Station', **SNOTEL_LINE)